In [1]:
import os
import pickle
import numpy as np
import datetime
from multiprocessing import Process

from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16, MobileNetV2

from models import *
from utils import regression_stats
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

2024-04-19 12:50:57.586286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 12:50:57.586352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 12:50:57.643871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-19 12:50:57.770869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 12:50:59.360346: W tensorflow/compiler/tf2

In [2]:
IMAGE_WIDTH: int = 448
IMAGE_HEIGHT: int = 448
MODELS_PATH: str = "./vision_models"
USE_GPU: bool = True

In [3]:
# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )
train_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train_prices: np.array = train1_df["price"].values

valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values

test_images: np.array = preprocess_images(
    test_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
test_prices: np.array = test_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 311/311 [00:00<00:00, 1926777.76it/s]


KeyboardInterrupt: 

In [ ]:
MODEL_NAME: str = "MobileNetV2"
FUNCTION: object = CNN_model
VARIABLES: tuple = (MobileNetV2, True, train_images, train_prices, valid_images, valid_prices)
# FUNCTION: object = N_CNN_model
# VARIABLES: tuple = (MobileNetV2, train_images, train_prices, valid_images, valid_prices, 3)

In [ ]:
def train_save_model(model_func: object, args: tuple, test_images: np.array, test_prices: np.array):
    """
    Train and save model
    
    Args:
        model_func (object): Model function
        args (tuple): Model arguments
    """
    if USE_GPU:
        set_gpu()
    else:
        set_cpu()

    model, fit_history = model_func(*args)

    print("Saving model...")
    # Save Model
    if not os.path.exists(f"{MODELS_PATH}/{MODEL_NAME}"):
        os.makedirs(f"{MODELS_PATH}/{MODEL_NAME}")
    model.save(f"{MODELS_PATH}/{MODEL_NAME}/model")

    # Save Training History
    with open(f"{MODELS_PATH}/{MODEL_NAME}/history", "wb") as file_pi:
        pickle.dump(fit_history.history, file_pi)

    # Export Model Architecture
    # plot_model(model, show_shapes=True, to_file=f"{MODELS_PATH}/{MODEL_NAME}/model_architecture.png")

    # Evaluate Model
    test_predictions = model.predict(test_images)
    r2, mae, percentage_error, mse = regression_stats(test_prices, test_predictions)
    feature_importance = model.feature_importance_
    # Export to txt file with timestamp
    with open(f"{MODELS_PATH}/{MODEL_NAME}/evaluation.txt", "w") as file:
        file.write(f"Timestamp: {datetime.datetime.now()}\n")
        file.write(f"R2: {r2}\n")
        file.write(f"MAE: {mae}\n")
        file.write(f"Percentage Error: {percentage_error}\n")
        file.write(f"MSE: {mse}\n")
        file.write("Feature Importances:\n")
        file.write(str(feature_importance))
        file.write("\n")


def train_save_models(model_func: object, args: tuple):
    """
    Train and save models (for n_cnn model)
    
    Args:
        model_func (object): Model function
        args (tuple): Model arguments
    """
    if USE_GPU:
        set_gpu()
    else:
        set_cpu()

    models, fit_histories = model_func(*args)

    print("Saving models...")
    for i, m in enumerate(models):
        # Save Model
        if not os.path.exists(f"{MODELS_PATH}/{MODEL_NAME}_{i}"):
            os.makedirs(f"{MODELS_PATH}/{MODEL_NAME}_{i}")
        m.save(f"{MODELS_PATH}/{MODEL_NAME}_{i}/model")

        # Save Training History
        with open(f"{MODELS_PATH}/{MODEL_NAME}_{i}/history", "wb") as file_pi:
            pickle.dump(fit_histories[i].history, file_pi)

        # Export Model Architecture
        # plot_model(m, show_shapes=True, to_file=f"{MODELS_PATH}/{MODEL_NAME}_{i}/model_architecture.png")
    

# p = Process(target=train_save_model, args=(FUNCTION, VARIABLES))
p = Process(target=train_save_models, args=(FUNCTION, VARIABLES))
p.start()
p.join()

2024-04-19 11:29:21.829395: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:21.856607: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:21.856714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:21.860799: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:21.860995: I external/local_xla/xla/stream_executor

1 Physical GPUs, 1 Logical GPU


2024-04-19 11:29:22.055156: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:22.055294: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:22.055310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-04-19 11:29:22.055371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 11:29:22.055392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

Epoch 1/150


2024-04-19 11:29:30.764769: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-19 11:29:30.974300: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-19 11:29:31.370199: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-19 11:29:35.325631: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fef81f05fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-19 11:29:35.325671: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-04-19 11:29:35.351511: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713518975.466593   69072 device_compiler.

7/7 [==============================] - 18s 1s/step - loss: 4187275.5000 - mean_absolute_error: 4185601.0000 - val_loss: 4403471.5000 - val_mean_absolute_error: 4402146.5000
Epoch 2/150
7/7 [==============================] - 3s 469ms/step - loss: 4177859.2500 - mean_absolute_error: 4176474.5000 - val_loss: 4377836.0000 - val_mean_absolute_error: 4375992.5000
Epoch 3/150
7/7 [==============================] - 2s 361ms/step - loss: 4128319.0000 - mean_absolute_error: 4125974.5000 - val_loss: 4257161.0000 - val_mean_absolute_error: 4253717.5000
Epoch 4/150
7/7 [==============================] - 2s 368ms/step - loss: 3922186.0000 - mean_absolute_error: 3917924.5000 - val_loss: 3817223.5000 - val_mean_absolute_error: 3811254.5000
Epoch 5/150
7/7 [==============================] - 2s 339ms/step - loss: 3229830.0000 - mean_absolute_error: 3222684.7500 - val_loss: 2561521.2500 - val_mean_absolute_error: 2552036.5000
Epoch 6/150
7/7 [==============================] - 2s 356ms/step - loss: 197221

Process Process-3:
Traceback (most recent call last):
  File "/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_67690/694297561.py", line 43, in train_save_models
    models, fit_histories = model_func(*args)
                            ^^^^^^^^^^^^^^^^^
  File "/mnt/c/Repos/house_price_regression/models.py", line 241, in N_CNN_model
    model, fit_history = CNN_model(
                         ^^^^^^^^^^
  File "/mnt/c/Repos/house_price_regression/models.py", line 211, in CNN_model
    fit_history = model.fit(
                  ^^^^^^^^^^
  File "/home/besplago/.pyenv/versions/penv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/hom